In [1]:
import pandas as pd
import numpy as np

# Load dataset

In [7]:
train = pd.read_csv("Data/Preprocessed/1-train-clean.csv")
val = pd.read_csv("Data/Preprocessed/2-val-clean.csv")
test = pd.read_csv("Data/Preprocessed/3-test-clean.csv")

X_train = train['review_clean'].tolist()
y_train = train.drop(columns=['review_clean'])

X_val = val['review_clean'].tolist()
y_val = val.drop(columns=['review_clean'])

X_test = test['review_clean'].tolist()
y_test = test.drop(columns=['review_clean'])

In [8]:
import random
random_indices = random.sample(list(y_train.index), 4)

y_train.loc[random_indices[0], 'food&drinks#miscellaneous'] = 1
y_train.loc[random_indices[1], 'room_amenities#miscellaneous'] = 1
y_train.loc[random_indices[2], 'rooms#miscellaneous'] = 1
y_train.loc[random_indices[3], 'room_amenities#prices'] = 1

# Score Definition

In [4]:
from sklearn.metrics import f1_score, classification_report

# Chuyển ma trận (N, K) thành ma trận nhị phân (N, 3*K) để tính F1-score
def multioutput_to_multilabel(y_sentiment_indices):
    if isinstance(y_sentiment_indices, pd.DataFrame):
        y_sentiment_indices = y_sentiment_indices.values

    nrow = y_sentiment_indices.shape[0] # Số lượng mẫu.
    ncol = y_sentiment_indices.shape[1] # Số lượng aspect.

    # Khởi tạo mảng Multi-label (Boolean) với kích thước: Hàng x (3 * Cột).
    multilabel = np.zeros((nrow, 3 * ncol), dtype=bool)
    for i in range(nrow):
        for j in range(ncol):
            sentiment_idx = y_sentiment_indices[i, j]
            if sentiment_idx != 0:
                pos = j * 3 + (sentiment_idx - 1)
                multilabel[i, pos] = True
    return multilabel

# Tính F1-score dựa trên ma trận nhị phân
def custom_f1_score(y_true, y_pred, average='micro', **kwargs):
    y_true_ml = multioutput_to_multilabel(y_true)
    y_pred_ml = multioutput_to_multilabel(y_pred)
    return round(f1_score(y_true_ml, y_pred_ml, average=average, **kwargs), 4)

# Tạo báo cáo phân loại dựa trên ma trận nhị phân
def custom_classification_report(y_true, y_pred, **kwargs):
    y_true_ml = multioutput_to_multilabel(y_true)
    y_pred_ml = multioutput_to_multilabel(y_pred)
    return classification_report(y_true_ml, y_pred_ml, **kwargs)

# **Machine Learning Model**

## TF-IDF

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=2, max_df=0.9)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

In [10]:
from sklearn.multioutput import MultiOutputClassifier as MOC

## PhoW2V Embedding

In [11]:
train_phow2v = pd.read_csv('Data/Embedding/1-train-phoW2V.csv')
val_phow2v = pd.read_csv('Data/Embedding/2-val-phoW2V.csv')
test_phow2v = pd.read_csv('Data/Embedding/3-test-phoW2V.csv')

In [13]:
X_train_phow2v = train_phow2v.iloc[:, :100]
X_val_phow2v = val_phow2v.iloc[:, :100]
X_test_phow2v = test_phow2v.iloc[:, :100]

## Search Hyperparameter Optimization

In [ ]:
import optuna
from optuna.samplers import TPESampler

d:\DS102\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key='best_model', value=trial.user_attrs['model'])

## Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegression

### TF-IDF

In [17]:
def logistic_objective(trial):
    params = dict(
        class_weight=trial.suggest_categorical('class_weight', ['balanced', None]),
        C=trial.suggest_float('C', 1e-5, 20),
        random_state=42,
        max_iter=200
    )    

    clf = MOC(LogisticRegression(**params))
    clf.fit(X_train_tfidf, y_train)
    trial.set_user_attr(key="model", value=clf)

    y_pred = clf.predict(X_val_tfidf)
    return custom_f1_score(y_val, y_pred)

sampler = TPESampler(seed=22)
logistic_study = optuna.create_study(sampler=sampler, direction='maximize')
logistic_study.optimize(logistic_objective, n_trials=20, callbacks=[callback])

lgr = logistic_study.user_attrs['best_model']

[I 2025-12-19 16:50:59,358] A new study created in memory with name: no-name-0aee7318-2a58-4a40-a941-28220fd7f967


[I 2025-12-19 16:51:05,287] Trial 0 finished with value: 0.6669 and parameters: {'class_weight': None, 'C': 8.41076650090714}. Best is trial 0 with value: 0.6669.
[I 2025-12-19 16:51:10,152] Trial 1 finished with value: 0.6882 and parameters: {'class_weight': 'balanced', 'C': 6.777285823434402}. Best is trial 1 with value: 0.6882.
[I 2025-12-19 16:51:15,723] Trial 2 finished with value: 0.6626 and parameters: {'class_weight': None, 'C': 4.408098128709346}. Best is trial 1 with value: 0.6882.
[I 2025-12-19 16:51:20,192] Trial 3 finished with value: 0.6864 and parameters: {'class_weight': 'balanced', 'C': 11.224078321223027}. Best is trial 1 with value: 0.6882.
[I 2025-12-19 16:51:24,697] Trial 4 finished with value: 0.6845 and parameters: {'class_weight': 'balanced', 'C': 3.7822352140976876}. Best is trial 1 with value: 0.6882.
[I 2025-12-19 16:51:30,464] Trial 5 finished with value: 0.6709 and parameters: {'class_weight': None, 'C': 19.156643783543824}. Best is trial 1 with value: 0.68

In [18]:
print("Logistic Regression with Best Hyperparameters F1-scores:")
print('train:', custom_f1_score(y_train, lgr.predict(X_train_tfidf)))
print('val:  ', custom_f1_score(y_val  , lgr.predict(X_val_tfidf)))
print('test: ', custom_f1_score(y_test , lgr.predict(X_test_tfidf)))

print(lgr.estimators_[0].get_params())
print("\nBest hyperparameters:")
print(logistic_study.best_params)

Logistic Regression with Best Hyperparameters F1-scores:
train: 0.9962
val:   0.6882
test:  0.7033
{'C': 6.777285823434402, 'class_weight': 'balanced', 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 200, 'multi_class': 'deprecated', 'n_jobs': None, 'penalty': 'l2', 'random_state': 42, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}

Best hyperparameters:
{'class_weight': 'balanced', 'C': 6.777285823434402}


### PhoW2V

In [19]:
def logistic_objective(trial):
    params = dict(
        class_weight=trial.suggest_categorical('class_weight', ['balanced', None]),
        C=trial.suggest_float('C', 1e-5, 20),
        random_state=42,
        max_iter=200
    )    

    clf = MOC(LogisticRegression(**params))
    clf.fit(X_train_phow2v, y_train)
    trial.set_user_attr(key="model", value=clf)

    y_pred = clf.predict(X_val_phow2v)
    return custom_f1_score(y_val, y_pred)

sampler = TPESampler(seed=22)
logistic_study = optuna.create_study(sampler=sampler, direction='maximize')
logistic_study.optimize(logistic_objective, n_trials=20, callbacks=[callback])

lgr = logistic_study.user_attrs['best_model']

[I 2025-12-19 16:53:59,949] A new study created in memory with name: no-name-3daacf80-1436-4b5f-8eb6-1fc20e89ffb9
[I 2025-12-19 16:54:00,694] Trial 0 finished with value: 0.625 and parameters: {'class_weight': None, 'C': 8.41076650090714}. Best is trial 0 with value: 0.625.
[I 2025-12-19 16:54:01,781] Trial 1 finished with value: 0.4352 and parameters: {'class_weight': 'balanced', 'C': 6.777285823434402}. Best is trial 0 with value: 0.625.
[I 2025-12-19 16:54:02,366] Trial 2 finished with value: 0.6136 and parameters: {'class_weight': None, 'C': 4.408098128709346}. Best is trial 0 with value: 0.625.
[I 2025-12-19 16:54:03,630] Trial 3 finished with value: 0.4562 and parameters: {'class_weight': 'balanced', 'C': 11.224078321223027}. Best is trial 0 with value: 0.625.
[I 2025-12-19 16:54:04,657] Trial 4 finished with value: 0.4087 and parameters: {'class_weight': 'balanced', 'C': 3.7822352140976876}. Best is trial 0 with value: 0.625.
[I 2025-12-19 16:54:05,446] Trial 5 finished with val

In [20]:
print("Logistic Regression with Best Hyperparameters F1-scores:")
print('train:', custom_f1_score(y_train, lgr.predict(X_train_phow2v)))
print('val:  ', custom_f1_score(y_val  , lgr.predict(X_val_phow2v)))
print('test: ', custom_f1_score(y_test , lgr.predict(X_test_phow2v)))

print(lgr.estimators_[0].get_params())
print("\nBest hyperparameters:")
print(logistic_study.best_params)

Logistic Regression with Best Hyperparameters F1-scores:
train: 0.6898
val:   0.6363
test:  0.6525
{'C': 19.156643783543824, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 200, 'multi_class': 'deprecated', 'n_jobs': None, 'penalty': 'l2', 'random_state': 42, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}

Best hyperparameters:
{'class_weight': None, 'C': 19.156643783543824}


## Linear SVC

In [21]:
from sklearn.svm import LinearSVC

### TF-IDF

In [ ]:
def linearsvc_objective(trial):
    params = dict(
        C=trial.suggest_float('C', 1e-9, 1e2, log=True),
        class_weight=trial.suggest_categorical('class_weight', ['balanced', None]),
        loss=trial.suggest_categorical('loss', ['hinge', 'squared_hinge']),
        max_iter=10000,
        random_state=42
    )
    
    if params['loss'] == 'hinge':
        params['dual'] = True
    clf = MOC(LinearSVC(**params))
    clf.fit(X_train_tfidf, y_train)
    trial.set_user_attr(key="model", value=clf)
    
    y_pred = clf.predict(X_val_tfidf)
    return custom_f1_score(y_val, y_pred)

sampler = TPESampler(seed=22)
svc_study = optuna.create_study(sampler=sampler, direction='maximize')
svc_study.optimize(linearsvc_objective, n_trials=20, callbacks=[callback])

svc_linear = svc_study.user_attrs['best_model']

[I 2025-12-19 15:40:07,661] A new study created in memory with name: no-name-edd6881a-ab5e-4fe5-a78d-75befb4cd290
[I 2025-12-19 15:40:07,911] Trial 0 finished with value: 0.4441 and parameters: {'C': 1.9636582699290402e-07, 'class_weight': 'balanced', 'loss': 'hinge'}. Best is trial 0 with value: 0.4441.
[I 2025-12-19 15:40:08,166] Trial 1 finished with value: 0.5715 and parameters: {'C': 5.339536586472381e-06, 'class_weight': None, 'loss': 'squared_hinge'}. Best is trial 1 with value: 0.5715.
[I 2025-12-19 15:40:08,341] Trial 2 finished with value: 0.5715 and parameters: {'C': 1.3055563380836963e-09, 'class_weight': None, 'loss': 'hinge'}. Best is trial 1 with value: 0.5715.
[I 2025-12-19 15:40:08,518] Trial 3 finished with value: 0.5715 and parameters: {'C': 1.1682869614143264e-09, 'class_weight': None, 'loss': 'hinge'}. Best is trial 1 with value: 0.5715.
[I 2025-12-19 15:40:08,911] Trial 4 finished with value: 0.6397 and parameters: {'C': 0.2804917948703948, 'class_weight': 'balanc

In [ ]:
print("Linear SVC with Best Hyperparameters F1-scores:")
print('train:', custom_f1_score(y_train, svc_linear.predict(X_train_tfidf)))
print('val:  ', custom_f1_score(y_val  , svc_linear.predict(X_val_tfidf)))
print('test: ', custom_f1_score(y_test , svc_linear.predict(X_test_tfidf)))

print(svc_linear.estimators_[0].get_params())
print("\nBest hyperparameters:")
print(svc_study.best_params)

Linear SVC with Best Hyperparameters F1-scores:
train: 0.9994
dev:   0.6827
test:  0.7016
{'C': 1.4791962969452819, 'class_weight': 'balanced', 'dual': 'auto', 'fit_intercept': True, 'intercept_scaling': 1, 'loss': 'squared_hinge', 'max_iter': 10000, 'multi_class': 'ovr', 'penalty': 'l2', 'random_state': 42, 'tol': 0.0001, 'verbose': 0}

Best hyperparameters:
{'C': 1.4791962969452819, 'class_weight': 'balanced', 'loss': 'squared_hinge'}


### PhoW2V

In [22]:
def linearsvc_objective(trial):
    params = dict(
        C=trial.suggest_float('C', 1e-9, 1e2, log=True),
        class_weight=trial.suggest_categorical('class_weight', ['balanced', None]),
        loss=trial.suggest_categorical('loss', ['hinge', 'squared_hinge']),
        max_iter=10000,
        random_state=42
    )
    
    if params['loss'] == 'hinge':
        params['dual'] = True
    clf = MOC(LinearSVC(**params))
    clf.fit(X_train_phow2v, y_train)
    trial.set_user_attr(key="model", value=clf)
    
    y_pred = clf.predict(X_val_phow2v)
    return custom_f1_score(y_val, y_pred)
sampler = TPESampler(seed=22)
svc_study = optuna.create_study(sampler=sampler, direction='maximize')
svc_study.optimize(linearsvc_objective, n_trials=20, callbacks=[callback])

svc_linear = svc_study.user_attrs['best_model']

[I 2025-12-19 16:55:58,061] A new study created in memory with name: no-name-cb389845-820a-4d9e-ac47-d71939ef6e24
[I 2025-12-19 16:55:58,375] Trial 0 finished with value: 0.4066 and parameters: {'C': 1.9636582699290402e-07, 'class_weight': 'balanced', 'loss': 'hinge'}. Best is trial 0 with value: 0.4066.
[I 2025-12-19 16:55:58,718] Trial 1 finished with value: 0.5715 and parameters: {'C': 5.339536586472381e-06, 'class_weight': None, 'loss': 'squared_hinge'}. Best is trial 1 with value: 0.5715.
[I 2025-12-19 16:55:58,963] Trial 2 finished with value: 0.5715 and parameters: {'C': 1.3055563380836963e-09, 'class_weight': None, 'loss': 'hinge'}. Best is trial 1 with value: 0.5715.
[I 2025-12-19 16:55:59,213] Trial 3 finished with value: 0.5715 and parameters: {'C': 1.1682869614143264e-09, 'class_weight': None, 'loss': 'hinge'}. Best is trial 1 with value: 0.5715.
[I 2025-12-19 16:56:01,277] Trial 4 finished with value: 0.4732 and parameters: {'C': 0.2804917948703948, 'class_weight': 'balanc

In [23]:
print("Linear SVC with Best Hyperparameters F1-scores:")
print('train:', custom_f1_score(y_train, svc_linear.predict(X_train_phow2v)))
print('val:  ', custom_f1_score(y_val  , svc_linear.predict(X_val_phow2v)))
print('test: ', custom_f1_score(y_test , svc_linear.predict(X_test_phow2v)))

print(svc_linear.estimators_[0].get_params())
print("\nBest hyperparameters:")
print(svc_study.best_params)

Linear SVC with Best Hyperparameters F1-scores:
train: 0.745
val:   0.6411
test:  0.6432
{'C': 78.77092791878185, 'class_weight': 'balanced', 'dual': 'auto', 'fit_intercept': True, 'intercept_scaling': 1, 'loss': 'squared_hinge', 'max_iter': 10000, 'multi_class': 'ovr', 'penalty': 'l2', 'random_state': 42, 'tol': 0.0001, 'verbose': 0}

Best hyperparameters:
{'C': 78.77092791878185, 'class_weight': 'balanced', 'loss': 'squared_hinge'}


## Non-Linear SVC

In [25]:
from sklearn.svm import SVC

### TF-IDF

In [ ]:
def nonlinear_svc_objective(trial):
    kernel_choice = trial.suggest_categorical('kernel', ['rbf', 'poly', 'sigmoid'])
    
    params = dict(
        C=trial.suggest_float('C', 1e-3, 100, log=True),
        kernel=kernel_choice,
        gamma=trial.suggest_categorical('gamma', ['scale', 'auto']), 
        class_weight=trial.suggest_categorical('class_weight', ['balanced', None]),
        random_state=42,
        max_iter=10000 
    )
    
    if kernel_choice == 'poly':
        params['degree'] = trial.suggest_int('degree', 2, 4)

    clf = MOC(SVC(**params))
    clf.fit(X_train_tfidf, y_train)
    trial.set_user_attr(key="model", value=clf)

    y_pred = clf.predict(X_val_tfidf)
    return custom_f1_score(y_val, y_pred)

sampler = TPESampler(seed=22)
svc_nonlinear_study = optuna.create_study(sampler=sampler, direction='maximize')
svc_nonlinear_study.optimize(nonlinear_svc_objective, n_trials=5, callbacks=[callback])

nonlinear_svc = svc_nonlinear_study.user_attrs['best_model']

[I 2025-12-19 15:56:13,649] A new study created in memory with name: no-name-36960476-171d-4d5c-ae07-8dfe26928a20
[I 2025-12-19 15:56:22,502] Trial 0 finished with value: 0.5715 and parameters: {'kernel': 'poly', 'C': 19.765599562374707, 'gamma': 'auto', 'class_weight': None, 'degree': 2}. Best is trial 0 with value: 0.5715.
[I 2025-12-19 15:57:16,970] Trial 1 finished with value: 0.6436 and parameters: {'kernel': 'rbf', 'C': 11.71199658483352, 'gamma': 'scale', 'class_weight': None}. Best is trial 1 with value: 0.6436.
[I 2025-12-19 15:58:24,242] Trial 2 finished with value: 0.6447 and parameters: {'kernel': 'rbf', 'C': 6.9177316302209935, 'gamma': 'scale', 'class_weight': 'balanced'}. Best is trial 2 with value: 0.6447.
[I 2025-12-19 15:59:40,218] Trial 3 finished with value: 0.5933 and parameters: {'kernel': 'poly', 'C': 41.338016019468874, 'gamma': 'scale', 'class_weight': 'balanced', 'degree': 2}. Best is trial 2 with value: 0.6447.
[I 2025-12-19 16:01:22,751] Trial 4 finished wit

In [ ]:
print("Non-Linear SVC with Best Hyperparameters F1-scores:")
print('train:', custom_f1_score(y_train, nonlinear_svc.predict(X_train_tfidf)))
print('val:  ', custom_f1_score(y_val  , nonlinear_svc.predict(X_val_tfidf)))
print('test: ', custom_f1_score(y_test , nonlinear_svc.predict(X_test_tfidf)))

print(nonlinear_svc.estimators_[0].get_params())
print("\nBest hyperparameters:")
print(svc_nonlinear_study.best_params)

Non-Linear SVC with Best Hyperparameters F1-scores:
train: 0.9996
dev:   0.6447
test:  0.6595
{'C': 6.9177316302209935, 'break_ties': False, 'cache_size': 200, 'class_weight': 'balanced', 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': 10000, 'probability': False, 'random_state': 42, 'shrinking': True, 'tol': 0.001, 'verbose': False}

Best hyperparameters:
{'kernel': 'rbf', 'C': 6.9177316302209935, 'gamma': 'scale', 'class_weight': 'balanced'}


### PhoW2V

In [26]:
def nonlinear_svc_objective(trial):
    kernel_choice = trial.suggest_categorical('kernel', ['rbf', 'poly', 'sigmoid'])
    
    params = dict(
        C=trial.suggest_float('C', 1e-3, 100, log=True),
        kernel=kernel_choice,
        gamma=trial.suggest_categorical('gamma', ['scale', 'auto']), 
        class_weight=trial.suggest_categorical('class_weight', ['balanced', None]),
        random_state=42,
        max_iter=10000 
    )
    
    if kernel_choice == 'poly':
        params['degree'] = trial.suggest_int('degree', 2, 4)

    clf = MOC(SVC(**params))
    clf.fit(X_train_phow2v, y_train)
    trial.set_user_attr(key="model", value=clf)

    y_pred = clf.predict(X_val_phow2v)
    return custom_f1_score(y_val, y_pred)

sampler = TPESampler(seed=22)
svc_nonlinear_study = optuna.create_study(sampler=sampler, direction='maximize')
svc_nonlinear_study.optimize(nonlinear_svc_objective, n_trials=5, callbacks=[callback])

nonlinear_svc = svc_nonlinear_study.user_attrs['best_model']

[I 2025-12-19 16:58:16,197] A new study created in memory with name: no-name-80b2ccf6-6b58-4b74-943c-67585c4c41ff
[I 2025-12-19 16:58:16,884] Trial 0 finished with value: 0.5715 and parameters: {'kernel': 'poly', 'C': 19.765599562374707, 'gamma': 'auto', 'class_weight': None, 'degree': 2}. Best is trial 0 with value: 0.5715.
[I 2025-12-19 16:58:18,394] Trial 1 finished with value: 0.6398 and parameters: {'kernel': 'rbf', 'C': 11.71199658483352, 'gamma': 'scale', 'class_weight': None}. Best is trial 1 with value: 0.6398.
[I 2025-12-19 16:58:21,153] Trial 2 finished with value: 0.5386 and parameters: {'kernel': 'rbf', 'C': 6.9177316302209935, 'gamma': 'scale', 'class_weight': 'balanced'}. Best is trial 1 with value: 0.6398.
[I 2025-12-19 16:58:22,788] Trial 3 finished with value: 0.5741 and parameters: {'kernel': 'poly', 'C': 41.338016019468874, 'gamma': 'scale', 'class_weight': 'balanced', 'degree': 2}. Best is trial 1 with value: 0.6398.
[I 2025-12-19 16:58:31,722] Trial 4 finished wit

In [27]:
print("Non-Linear SVC with Best Hyperparameters F1-scores:")
print('train:', custom_f1_score(y_train, nonlinear_svc.predict(X_train_phow2v)))
print('val:  ', custom_f1_score(y_val  , nonlinear_svc.predict(X_val_phow2v)))
print('test: ', custom_f1_score(y_test , nonlinear_svc.predict(X_test_phow2v)))

print(nonlinear_svc.estimators_[0].get_params())
print("\nBest hyperparameters:")
print(svc_nonlinear_study.best_params)

Non-Linear SVC with Best Hyperparameters F1-scores:
train: 0.7073
val:   0.6398
test:  0.6436
{'C': 11.71199658483352, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': 10000, 'probability': False, 'random_state': 42, 'shrinking': True, 'tol': 0.001, 'verbose': False}

Best hyperparameters:
{'kernel': 'rbf', 'C': 11.71199658483352, 'gamma': 'scale', 'class_weight': None}


## MultinomialNB

In [31]:
from sklearn.naive_bayes import MultinomialNB

### TF-IDF

In [ ]:
def multinomial_nb_objective(trial):
    params = dict(
        alpha=trial.suggest_float('alpha', 1e-3, 10, log=True),
        fit_prior=trial.suggest_categorical('fit_prior', [True, False])
    )
    
    clf = MOC(MultinomialNB(**params))
    clf.fit(X_train_tfidf, y_train)
    trial.set_user_attr(key="model", value=clf)

    y_pred = clf.predict(X_val_tfidf)
    return custom_f1_score(y_val, y_pred)

sampler = TPESampler(seed=22)
nb_study = optuna.create_study(sampler=sampler, direction='maximize')
nb_study.optimize(multinomial_nb_objective, n_trials=50, callbacks=[callback])

nb_model = nb_study.user_attrs['best_model']

[I 2025-12-19 16:11:31,828] A new study created in memory with name: no-name-71318f66-9f15-4a1d-87ba-b831f676bc14
[I 2025-12-19 16:11:32,132] Trial 0 finished with value: 0.6179 and parameters: {'alpha': 0.006820907334120959, 'fit_prior': True}. Best is trial 0 with value: 0.6179.
[I 2025-12-19 16:11:32,236] Trial 1 finished with value: 0.583 and parameters: {'alpha': 2.733556118022411, 'fit_prior': False}. Best is trial 0 with value: 0.6179.
[I 2025-12-19 16:11:32,340] Trial 2 finished with value: 0.6247 and parameters: {'alpha': 0.012081791403069187, 'fit_prior': True}. Best is trial 2 with value: 0.6247.
[I 2025-12-19 16:11:32,436] Trial 3 finished with value: 0.5888 and parameters: {'alpha': 1.7693089816650007, 'fit_prior': False}. Best is trial 2 with value: 0.6247.
[I 2025-12-19 16:11:32,537] Trial 4 finished with value: 0.5747 and parameters: {'alpha': 1.7984764264034472, 'fit_prior': True}. Best is trial 2 with value: 0.6247.
[I 2025-12-19 16:11:32,634] Trial 5 finished with va

In [ ]:
print("MultinomialNB with Best Hyperparameters F1-scores:")
print('train:', custom_f1_score(y_train, nb_model.predict(X_train_tfidf)))
print('val:  ', custom_f1_score(y_val  , nb_model.predict(X_val_tfidf)))
print('test: ', custom_f1_score(y_test , nb_model.predict(X_test_tfidf)))

print(nb_model.estimators_[0].get_params())
print("\nBest hyperparameters:")
print(nb_study.best_params)

MultinomialNB with Best Hyperparameters F1-scores:
train: 0.9779
dev:   0.6303
test:  0.6308
{'alpha': 0.051433368551114446, 'class_prior': None, 'fit_prior': True, 'force_alpha': True}

Best hyperparameters:
{'alpha': 0.051433368551114446, 'fit_prior': True}


## Random Forest

In [33]:
from sklearn.ensemble import RandomForestClassifier

### TF-IDF

In [ ]:
def rf_objective(trial):
    params = dict(
        n_estimators=trial.suggest_int('n_estimators', 50, 200),
        max_depth=trial.suggest_int('max_depth', 10, 100),
        min_samples_split=trial.suggest_int('min_samples_split', 2, 15),
        min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 10),
        class_weight=trial.suggest_categorical('class_weight', ['balanced', 'balanced_subsample', None]),
        n_jobs=-1, 
        random_state=42
    )
    
    clf = MOC(RandomForestClassifier(**params))
    clf.fit(X_train_tfidf, y_train)
    trial.set_user_attr(key="model", value=clf)

    y_pred = clf.predict(X_val_tfidf)
    return custom_f1_score(y_val, y_pred)

sampler = TPESampler(seed=22)
rf_study = optuna.create_study(sampler=sampler, direction='maximize')
rf_study.optimize(rf_objective, n_trials=10, callbacks=[callback])

rf_model = rf_study.user_attrs['best_model']

[I 2025-12-19 16:11:46,727] A new study created in memory with name: no-name-55d69582-7cbe-4480-b0be-dfb5a1c650dd
[I 2025-12-19 16:11:54,833] Trial 0 finished with value: 0.6408 and parameters: {'n_estimators': 81, 'max_depth': 53, 'min_samples_split': 7, 'min_samples_leaf': 9, 'class_weight': 'balanced_subsample'}. Best is trial 0 with value: 0.6408.
[I 2025-12-19 16:12:10,664] Trial 1 finished with value: 0.648 and parameters: {'n_estimators': 154, 'max_depth': 30, 'min_samples_split': 13, 'min_samples_leaf': 1, 'class_weight': 'balanced_subsample'}. Best is trial 1 with value: 0.648.
[I 2025-12-19 16:12:16,872] Trial 2 finished with value: 0.5911 and parameters: {'n_estimators': 78, 'max_depth': 10, 'min_samples_split': 12, 'min_samples_leaf': 10, 'class_weight': None}. Best is trial 1 with value: 0.648.
[I 2025-12-19 16:12:32,252] Trial 3 finished with value: 0.6723 and parameters: {'n_estimators': 153, 'max_depth': 45, 'min_samples_split': 10, 'min_samples_leaf': 5, 'class_weight'

In [ ]:
print("RandomForestClassifier with Best Hyperparameters F1-scores:")
print('train:', custom_f1_score(y_train, rf_model.predict(X_train_tfidf)))
print('val:  ', custom_f1_score(y_val  , rf_model.predict(X_val_tfidf)))
print('test: ', custom_f1_score(y_test , rf_model.predict(X_test_tfidf)))

print(rf_model.estimators_[0].get_params())
print("\nBest hyperparameters:")
print(rf_study.best_params)

RandomForestClassifier with Best Hyperparameters F1-scores:
train: 0.9297
dev:   0.6723
test:  0.6878
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': 'balanced_subsample', 'criterion': 'gini', 'max_depth': 45, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 5, 'min_samples_split': 10, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 153, 'n_jobs': -1, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}

Best hyperparameters:
{'n_estimators': 153, 'max_depth': 45, 'min_samples_split': 10, 'min_samples_leaf': 5, 'class_weight': 'balanced_subsample'}


# **PhoBERT**

In [ ]:
import torch
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoConfig
import torch.nn as nn
from torch.optim import AdamW
from tqdm.auto import tqdm

## VLSP2018 DataLoader

In [ ]:
import re
import csv
from tqdm import tqdm
from datasets import load_dataset


class PolarityMapping:
    INDEX_TO_POLARITY = { 0: None, 1: 'positive', 2: 'negative', 3: 'neutral' }
    INDEX_TO_ONEHOT = { 0: [1, 0, 0, 0], 1: [0, 1, 0, 0], 2: [0, 0, 1, 0], 3: [0, 0, 0, 1] }
    POLARITY_TO_INDEX = { None: 0, 'positive': 1, 'negative': 2, 'neutral': 3 }


class VLSP2018Loader:

    @staticmethod
    def load(train_csv_path, val_csv_path, test_csv_path):
        dataset_paths = {'train': train_csv_path, 'val': val_csv_path, 'test': test_csv_path}
        raw_datasets = load_dataset('csv', data_files={ k: v for k, v in dataset_paths.items() if v })
        return raw_datasets


    @staticmethod
    def preprocess_and_tokenize(text_data, preprocessor, tokenizer, batch_size, max_length):
        print('[INFO] Preprocessing and tokenizing text data...')
        def transform_each_batch(batch):
            preprocessed_batch = preprocessor.process_batch(batch)
            return tokenizer(preprocessed_batch, max_length=max_length, padding='max_length', truncation=True)

        if type(text_data) == str: return transform_each_batch([text_data])
        return text_data.map(
            lambda reviews: transform_each_batch(reviews['Review']),
            batched=True, batch_size=batch_size
        ).remove_columns('Review')


    @staticmethod
    def labels_to_flatten_onehot(datasets):
        print('[INFO] Transforming "Aspect#Categoy,Polarity" labels to flattened one-hot encoding...')
        model_input_names = ['input_ids', 'token_type_ids', 'attention_mask']
        label_columns = [col for col in datasets['train'].column_names if col not in ['Review', *model_input_names]]
        def transform_each_review(review): # Convert each Aspect#Categoy,Polarity to one-hot encoding and merge them into 1D list
            review['FlattenOneHotLabels'] = sum([
                PolarityMapping.INDEX_TO_ONEHOT[review[aspect_category]] # Get one-hot encoding
                for aspect_category in label_columns
            ], []) # Need to be flattened to match the model's output shape
            return review
        return datasets.map(transform_each_review, num_proc=8).select_columns(['FlattenOneHotLabels', *model_input_names])


class VLSP2018Parser:
    def __init__(self, train_txt_path, val_txt_path=None, test_txt_path=None):
        self.dataset_paths = { 'train': train_txt_path, 'val': val_txt_path, 'test': test_txt_path }
        self.reviews = { 'train': [], 'val': [], 'test': [] }
        self.aspect_categories = set()

        for dataset_type, txt_path in self.dataset_paths.items():
            if not txt_path:
                self.dataset_paths.pop(dataset_type)
                self.reviews.pop(dataset_type)
        self._parse_input_files()


    def _parse_input_files(self):
        print(f'[INFO] Parsing {len(self.dataset_paths)} input files...')
        for dataset_type, txt_path in self.dataset_paths.items():
            with open(txt_path, 'r', encoding='utf-8') as txt_file:
                content = txt_file.read()
                review_blocks = content.strip().split('\n\n')

                for block in tqdm(review_blocks):
                    lines = block.split('\n')
                    sentiment_info = re.findall(r'\{([^,]+)#([^,]+), ([^}]+)\}', lines[2].strip())

                    review_data = {}
                    for aspect, category, polarity in sentiment_info:
                        aspect_category = f'{aspect.strip()}#{category.strip()}'
                        self.aspect_categories.add(aspect_category)
                        review_data[aspect_category] = PolarityMapping.POLARITY_TO_INDEX[polarity.strip()]

                    self.reviews[dataset_type].append((lines[1].strip(), review_data))
        self.aspect_categories = sorted(self.aspect_categories)


    def txt2csv(self):
        print('[INFO] Converting parsed data to CSV files...')
        for dataset, txt_path in self.dataset_paths.items():
            csv_path = txt_path.replace('.txt', '.csv')

            with open(csv_path, 'w', newline='', encoding='utf-8') as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow(['Review'] + self.aspect_categories)

                for review_text, review_data in tqdm(self.reviews[dataset]):
                    row = [review_text] + [review_data.get(aspect_category, 0) for aspect_category in self.aspect_categories]
                    writer.writerow(row)

    @staticmethod
    def vlsp_save_as(save_path, raw_texts, encoded_review_labels, aspect_category_names):
        with open(save_path, 'w', encoding='utf-8') as file:
            for index, encoded_label in tqdm(enumerate(encoded_review_labels)):
                polarities = map(lambda x: PolarityMapping.INDEX_TO_POLARITY[x], encoded_label)
                acsa = ', '.join(
                    f'{{{aspect_category}, {polarity}}}'
                    for aspect_category, polarity in zip(aspect_category_names, polarities) if polarity
                )
                file.write(f"#{index + 1}\n{raw_texts[index]}\n{acsa}\n\n")


## Config

In [ ]:
TRAIN_PATH = r'Data/Preprocessed/1-train-clean.csv'
VAL_PATH = r'Data/Preprocessed/2-val-clean.csv'
TEST_PATH = r'Data/Preprocessed/3-test-clean.csv'
PRETRAINED_MODEL = 'vinai/phobert-base'
MAX_LENGTH = 256
BATCH_SIZE = 16
EPOCHS = 30
NUM_CLASSES = 4 # Giả định: 0: None, 1: Pos, 2: Neg, 3: Neu

## Load Data

In [ ]:
raw_datasets = VLSP2018Loader.load(TRAIN_PATH, VAL_PATH, TEST_PATH)
raw_datasets

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['review_clean', 'hotel#general', 'hotel#prices', 'hotel#design&features', 'hotel#cleanliness', 'hotel#comfort', 'hotel#quality', 'hotel#miscellaneous', 'rooms#general', 'rooms#prices', 'rooms#design&features', 'rooms#cleanliness', 'rooms#comfort', 'rooms#quality', 'rooms#miscellaneous', 'room_amenities#general', 'room_amenities#prices', 'room_amenities#design&features', 'room_amenities#cleanliness', 'room_amenities#comfort', 'room_amenities#quality', 'room_amenities#miscellaneous', 'facilities#general', 'facilities#prices', 'facilities#design&features', 'facilities#cleanliness', 'facilities#comfort', 'facilities#quality', 'facilities#miscellaneous', 'service#general', 'location#general', 'food&drinks#prices', 'food&drinks#quality', 'food&drinks#style&options', 'food&drinks#miscellaneous'],
        num_rows: 1658
    })
    val: Dataset({
        features: ['review_clean', 'hotel#general', 'hotel#prices', 'hotel#design&features', 'ho

In [ ]:
ignore_cols = ['review_clean']
label_cols = [col for col in raw_datasets['train'].column_names if col not in ignore_cols]
print(f"Số lượng Aspects cần dự đoán: {len(label_cols)}")

Số lượng Aspects cần dự đoán: 34


## Tokenize

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

def tokenize_function(examples):
    return tokenizer(
        examples['review_clean'],
        padding='max_length',
        truncation=True,
        max_length=MAX_LENGTH
    )

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/1658 [00:00<?, ? examples/s]

Map:   0%|          | 0/359 [00:00<?, ? examples/s]

Map:   0%|          | 0/372 [00:00<?, ? examples/s]

In [ ]:
# --- 4. FORMAT CHO PYTORCH ---
# Xóa cột text gốc để nhẹ bộ nhớ
tokenized_datasets = tokenized_datasets.remove_columns(['review_clean'])

# Định nghĩa các cột cần giữ lại (Gồm input của model + các cột label gốc)
input_cols = ['input_ids', 'attention_mask', 'token_type_ids']
cols_to_keep = input_cols + label_cols

# Quan trọng: Chuyển toàn bộ sang Tensor
tokenized_datasets.set_format("torch", columns=cols_to_keep)

# Tạo DataLoader
train_dataloader = DataLoader(tokenized_datasets['train'], shuffle=True, batch_size=BATCH_SIZE)
val_dataloader = DataLoader(tokenized_datasets['val'], batch_size=BATCH_SIZE)
test_dataloader = DataLoader(tokenized_datasets['test'], batch_size=BATCH_SIZE)

## Model

In [ ]:
class PhoBertABSA(nn.Module):
    def __init__(self, model_name, aspect_names, num_classes):
        super().__init__()
        self.phobert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)

        # Dùng ModuleDict để lưu các layer theo tên cột (không cần đổi tên)
        # Ví dụ: self.classifiers['hotel#prices'] là một lớp Linear riêng
        self.classifiers = nn.ModuleDict({
            name: nn.Linear(768, num_classes) for name in aspect_names
        })

    def forward(self, input_ids, attention_mask, token_type_ids):
        # 1. Chạy PhoBERT
        outputs = self.phobert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        # Lấy vector đặc trưng của câu (CLS token)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)

        # 2. Chạy qua từng nhánh aspect
        logits_dict = {}
        for name, classifier in self.classifiers.items():
            logits_dict[name] = classifier(pooled_output)

        return logits_dict

### Train loop

In [ ]:
def train_loop(model, dataloader, optimizer, device, loss_fn):
    model.train()
    total_loss = 0

    progress_bar = tqdm(dataloader, desc="Training")

    for batch in progress_bar:
        # Đẩy input vào GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)

        # Reset gradient
        optimizer.zero_grad()

        # Forward pass (Model dự đoán)
        logits_dict = model(input_ids, attention_mask, token_type_ids)

        # Tính Loss tổng
        batch_loss = 0
        for aspect_name in label_cols:
            # Lấy nhãn thật của aspect này từ batch
            labels = batch[aspect_name].to(device)

            # Lấy dự đoán của aspect này
            logits = logits_dict[aspect_name]

            # Cộng dồn loss (CrossEntropy)
            batch_loss += loss_fn(logits, labels)

        # Backward pass
        batch_loss.backward()
        optimizer.step()

        total_loss += batch_loss.item()
        progress_bar.set_postfix({'loss': batch_loss.item()})

    return total_loss / len(dataloader)

## Main

In [ ]:
# 1. Setup thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. Khởi tạo Model
model = PhoBertABSA(PRETRAINED_MODEL, label_cols, NUM_CLASSES)
model.to(device)

# 3. Setup Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss()

# 4. Bắt đầu Train
print("Start Training...")
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")

    train_loss = train_loop(model, train_dataloader, optimizer, device, loss_fn)
    print(f"Average Train Loss: {train_loss:.4f}")

Using device: cuda


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Start Training...

Epoch 1/30


Training:   0%|          | 0/104 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

...

Average Train Loss: 1.7503

Epoch 29/30


Training: 100%|██████████| 104/104 [01:09<00:00,  1.49it/s, loss=2.04]


Average Train Loss: 1.6553

Epoch 30/30


Training: 100%|██████████| 104/104 [01:09<00:00,  1.50it/s, loss=3.33]

Average Train Loss: 1.5664


## Evaluation

In [ ]:
def get_predictions(model, dataloader, device, aspect_names):
    model.eval() # Chuyển sang chế độ đánh giá (tắt dropout)

    all_preds = []
    all_labels = []

    # Không tính gradient để tiết kiệm bộ nhớ
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            # 1. Đẩy input vào GPU
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)

            # 2. Model dự đoán
            logits_dict = model(input_ids, attention_mask, token_type_ids)

            # 3. Xử lý kết quả từng Batch
            batch_preds = []
            batch_labels = []

            # Duyệt qua từng aspect theo đúng thứ tự trong label_cols
            for aspect in aspect_names:
                # --- Xử lý Prediction ---
                # Lấy logits của aspect đó
                logits = logits_dict[aspect]
                # Chọn class có xác suất cao nhất (argmax)
                preds = logits.argmax(dim=-1)
                batch_preds.append(preds.cpu().numpy())

                # --- Xử lý Label thật ---
                labels = batch[aspect]
                batch_labels.append(labels.cpu().numpy())

            # batch_preds đang là list các array [Batch_Size], ta cần stack lại
            # Kết quả: mảng (Batch_Size, Num_Aspects)
            all_preds.append(np.stack(batch_preds, axis=1))
            all_labels.append(np.stack(batch_labels, axis=1))

    # Nối tất cả các batch lại thành một ma trận lớn
    # Shape cuối cùng: (Total_Samples, Num_Aspects)
    final_preds = np.concatenate(all_preds, axis=0)
    final_labels = np.concatenate(all_labels, axis=0)

    return final_labels, final_preds

In [ ]:
# --- 1. Đánh giá trên tập Validation (Val) ---
print("Đang chạy dự đoán trên tập Validation...")
y_true_val, y_pred_val = get_predictions(model, val_dataloader, device, label_cols)

val_micro_f1 = custom_f1_score(y_true_val, y_pred_val)

print(f"Kích thước ma trận Val: {y_pred_val.shape}")
print("-" * 30)
print(f"Kết quả đánh giá trên tập VALIDATION:")
print(f"Micro F1-score: {val_micro_f1}")
print("-" * 30)

# --- 2. Đánh giá trên tập Test ---
print("\nĐang chạy dự đoán trên tập Test...")
y_true_test, y_pred_test = get_predictions(model, test_dataloader, device, label_cols)

test_micro_f1 = custom_f1_score(y_true_test, y_pred_test)

print(f"Kích thước ma trận Test: {y_pred_test.shape}")
print("-" * 30)
print(f"Kết quả đánh giá trên tập TEST:")
print(f"Micro F1-score: {test_micro_f1}")
print("-" * 30)

Đang chạy dự đoán trên tập Validation...


Evaluating: 100%|██████████| 23/23 [00:04<00:00,  5.18it/s]


Kích thước ma trận Val: (359, 34)
------------------------------
Kết quả đánh giá trên tập VALIDATION:
Micro F1-score: 0.7229
------------------------------

Đang chạy dự đoán trên tập Test...


Evaluating: 100%|██████████| 24/24 [00:04<00:00,  5.37it/s]

Kích thước ma trận Test: (372, 34)
------------------------------
Kết quả đánh giá trên tập TEST:
Micro F1-score: 0.7383
------------------------------
